Mount Google Drive

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')
!ls "/content/gdrive/My Drive/SML_Project1"

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive
 checkpoint
'Copy of tokenizer.ipynb'
 Data.ipynb
 mlp2.hdf5
 mlp3.hdf5
 mlp3.png
 mlp4.hdf5
 mlp4.png
 mlp5.hdf5
 mlp.hdf5
 mlp_reg.hdf5
 mlp_sgd_30+.png
 mlp_sgd.hdf5
 mlp_sgd.png
'Pre-trained BERT contextualized word embeddings.ipynb'
 pridicted.csv
 pridicted.gsheet
 project1.ipynb
 test_tweets_unlabeled.txt
 tokenizer.ipynb
 train_tweets.txt
 wwm_uncased_L-24_H-1024_A-16
 x_train_encode.json


Clone bert from github

In [2]:
!rm -rf bert
!git clone https://github.com/google-research/bert

Cloning into 'bert'...
remote: Enumerating objects: 333, done.
remote: Total 333 (delta 0), reused 0 (delta 0), pack-reused 333
Receiving objects: 100% (333/333), 282.45 KiB | 3.87 MiB/s, done.
Resolving deltas: 100% (183/183), done.


In [0]:
import sys

sys.path.append('bert/')

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import codecs
import collections
import json
import re
import os
import pprint
import numpy as np
import tensorflow as tf

import modeling
import tokenization

Configure TPU

In [4]:
assert 'COLAB_TPU_ADDR' in os.environ, 'ERROR: Not connected to a TPU runtime; please see the first cell in this notebook for instructions!'
TPU_ADDRESS = 'grpc://' + os.environ['COLAB_TPU_ADDR']
print('TPU address is', TPU_ADDRESS)

from google.colab import auth
auth.authenticate_user()
with tf.Session(TPU_ADDRESS) as session:
  print('TPU devices:')
  pprint.pprint(session.list_devices())

  # Upload credentials to TPU.
  with open('/content/adc.json', 'r') as f:
    auth_info = json.load(f)
  tf.contrib.cloud.configure_gcs(session, credentials=auth_info)
  # Now credentials are set for all future sessions on this TPU.

TPU address is grpc://10.43.48.66:8470
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

TPU devices:
[_DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:CPU:0, CPU, -1, 11183142539871364756),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 17179869184, 474250815191198440),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:0, TPU, 17179869184, 17588290432072764636),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:1, TPU, 17179869184, 7538589241765838510),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:2, TPU, 17179869184, 3977842903233554643),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/de

Lemmatization

In [5]:
import nltk
nltk.download('wordnet')
lemmatizer = nltk.stem.wordnet.WordNetLemmatizer()

def lemmatize(word):
    lemma = lemmatizer.lemmatize(word,'v')
    if lemma == word:
        lemma = lemmatizer.lemmatize(word,'n')
    return lemma


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


Tokenizer and extract features

In [0]:
import re
from nltk.stem.porter import *

rt_str = r'RT'
capital_str = r'[A-Z]'
mention_str = r'@handle(:)?'
emoticons_str = r'[:=;][oO\-]?[D\)\]\(\]/\\OpP]'         
http_str = r'([-|:] )?http[s]?://(?:[a-z]|[0-9]|[$-_@.&amp;+]|[!*\(\),]|(?:%[0-9a-f][0-9a-f]))+'
#hashtag_str = r"(?:\#+[\w_]+[\w\'_\-]*[\w]+)"
hashtag_str = r"#"
money_str = r"$[0-9]+"
percentage_str  = r'[0-9]+%'

feature_regex_strs = [
    http_str,
    rt_str,
    emoticons_str,
    mention_str,
    hashtag_str,
    money_str,
    percentage_str
]

'''
punctuation_strs = [
    r'\.',
    r',',
    r'!',
    r'\?',
    r':',
    r';',
    r'\'',
    r'\"',
    r'<[^>]+>',
    r'\([^>]+\)',
    r'\[[^>]+\]',
    r'\{[^>]+\}',
    r'[\^|\*|_|\-|=|\+|\/|\\|\||`|~|&]'
]

feature_regex_strs.extend(punctuation_strs)
'''
class Tokenizer():
    """
    This class is used to tokenize the tweets and calculate the average length.
    feature vector: number of url, number of 'RT', number of emotions, number of @, number of hashtag, number of cash mentioned, number of percentage, fraction of capitals
    """
    def __init__(self):
        self.processed_length = 0
        self.processed_item = 1
        self.features = []
        self.max_seq = 0
        self.vocab = {}
        
    def num_of_match(self, patten, tweet):
        return len(re.findall(patten, tweet))

    def tokenize(self, tweets):
        """
        :param tweets: One tweet
        :return: tokens: tokenized tweet
                 feature array including captical number and @ number
        """
        
        other_features = []
        for feature_regex_str in feature_regex_strs:
            other_features.append(self.num_of_match(feature_regex_str, tweets))
            tweets = re.sub(feature_regex_str, '', tweets)
        
        other_features.append(float(self.num_of_match(capital_str, tweets)) / len(tweets))
        
        self.features.append(other_features)

        regex_str = [
            r'(?:(?:\d+,?)+(?:\.?\d+)?)',  # number
            r"(?:[a-z][a-z'\-_]+[a-z])",  # word with - and '
            r'(?:[\w_]+)',
            r'(?:\S)'
        ]

        tokens_re = re.compile(r'(' + '|'.join(regex_str) + ')', re.VERBOSE | re.IGNORECASE)

        tokens = tokens_re.findall(tweets)
        tokens = [token.lower() if token.isalpha() else token for token in tokens]
        tokens = [lemmatize(token) if token.isalpha() else token for token in tokens]
        
        token_length = 0
        for token in tokens:
            token_length += len(token)
            if token not in self.vocab:
                self.vocab[token] = len(self.vocab)
        self.processed_length += token_length
        self.processed_item += 1
        self.max_seq = max(self.max_seq, len(tokens))
        return tokens
      
    def avg_length(self):
        """
        :return: Average length of tokens
        """
        return float(self.processed_length) / self.processed_item 
      
    def get_other_features(self):
        return self.features
      
    def get_max_seq(self):
        return self.max_seq
    
    def get_vocab(self):
        return self.vocab
      
mytokenizer = Tokenizer()

#tweet_example = 'RT @handle: Cool SEO post by @handle ! :) #RRPP #PR RT @handle: Top 10 #SEO Tips for #Public Relations - http://ow.ly/Bh7L'
#tweet_example2 = 'RT @handle: Note to webmasters: the full roll out of Caffeine won\'t happen until after the holidays. More info: http://bit.ly/4GELv6s'
#tokens_example = mytokenizer.tokenize(tweet_example)
#tokens_example2 = mytokenizer.tokenize(tweet_example2)
#print(tokens_example)
#print(tokens_example2)
#print(mytokenizer.get_other_features())



Pretrain BERT configure

In [7]:
# Available pretrained model checkpoints:
#   uncased_L-12_H-768_A-12: uncased BERT base model
#   uncased_L-24_H-1024_A-16: uncased BERT large model
#   cased_L-12_H-768_A-12: cased BERT large model
BERT_MODEL = 'uncased_L-12_H-768_A-12' #@param {type:"string"}
BERT_PRETRAINED_DIR = 'gs://cloud-tpu-checkpoints/bert/' + BERT_MODEL
print('***** BERT pretrained directory: {} *****'.format(BERT_PRETRAINED_DIR))
!gsutil ls $BERT_PRETRAINED_DIR

***** BERT pretrained directory: gs://cloud-tpu-checkpoints/bert/uncased_L-12_H-768_A-12 *****
gs://cloud-tpu-checkpoints/bert/uncased_L-12_H-768_A-12/bert_config.json
gs://cloud-tpu-checkpoints/bert/uncased_L-12_H-768_A-12/bert_model.ckpt.data-00000-of-00001
gs://cloud-tpu-checkpoints/bert/uncased_L-12_H-768_A-12/bert_model.ckpt.index
gs://cloud-tpu-checkpoints/bert/uncased_L-12_H-768_A-12/bert_model.ckpt.meta
gs://cloud-tpu-checkpoints/bert/uncased_L-12_H-768_A-12/checkpoint
gs://cloud-tpu-checkpoints/bert/uncased_L-12_H-768_A-12/vocab.txt


In [8]:
LAYERS = [-1,-2,-3,-4]
NUM_TPU_CORES = 8
MAX_SEQ_LENGTH = 140
BERT_CONFIG = BERT_PRETRAINED_DIR + '/bert_config.json'
CHKPT_DIR = BERT_PRETRAINED_DIR + '/bert_model.ckpt'
VOCAB_FILE = BERT_PRETRAINED_DIR + '/vocab.txt'
INIT_CHECKPOINT = BERT_PRETRAINED_DIR + '/bert_model.ckpt'
BATCH_SIZE = 128
dictionary = tokenization.load_vocab(VOCAB_FILE)

Create an input example

In [0]:
class InputExample(object):

  def __init__(self, unique_id, text_a, text_b=None):
    self.unique_id = unique_id
    self.text_a = text_a
    self.text_b = text_b


In [0]:
class InputFeatures(object):
  """A single set of features of data."""

  def __init__(self, unique_id, tokens, input_ids, input_mask, input_type_ids):
    self.unique_id = unique_id
    self.tokens = tokens
    self.input_ids = input_ids
    self.input_mask = input_mask
    self.input_type_ids = input_type_ids

Reformate input for pretrained BERT 

In [0]:
def input_fn_builder(features, seq_length):
  """Creates an `input_fn` closure to be passed to TPUEstimator."""

  all_unique_ids = []
  all_input_ids = []
  all_input_mask = []
  all_input_type_ids = []

  for feature in features:
    all_unique_ids.append(feature.unique_id)
    all_input_ids.append(feature.input_ids)
    all_input_mask.append(feature.input_mask)
    all_input_type_ids.append(feature.input_type_ids)

  def input_fn(params):
    """The actual input function."""
    batch_size = params["batch_size"]

    num_examples = len(features)

    # This is for demo purposes and does NOT scale to large data sets. We do
    # not use Dataset.from_generator() because that uses tf.py_func which is
    # not TPU compatible. The right way to load data is with TFRecordReader.
    d = tf.data.Dataset.from_tensor_slices({
        "unique_ids":
            tf.constant(all_unique_ids, shape=[num_examples], dtype=tf.int32),
        "input_ids":
            tf.constant(
                all_input_ids, shape=[num_examples, seq_length],
                dtype=tf.int32),
        "input_mask":
            tf.constant(
                all_input_mask,
                shape=[num_examples, seq_length],
                dtype=tf.int32),
        "input_type_ids":
            tf.constant(
                all_input_type_ids,
                shape=[num_examples, seq_length],
                dtype=tf.int32),
    })

    d = d.batch(batch_size=batch_size, drop_remainder=False)
    return d

  return input_fn

Build prettrained BERT model

In [0]:
def model_fn_builder(bert_config, init_checkpoint, layer_indexes, use_tpu,
                     use_one_hot_embeddings):
  """Returns `model_fn` closure for TPUEstimator."""

  def model_fn(features, labels, mode, params):  # pylint: disable=unused-argument
    """The `model_fn` for TPUEstimator."""

    unique_ids = features["unique_ids"]
    input_ids = features["input_ids"]
    input_mask = features["input_mask"]
    input_type_ids = features["input_type_ids"]

    model = modeling.BertModel(
        config=bert_config,
        is_training=False,
        input_ids=input_ids,
        input_mask=input_mask,
        token_type_ids=input_type_ids,
        use_one_hot_embeddings=use_one_hot_embeddings)

    if mode != tf.estimator.ModeKeys.PREDICT:
      raise ValueError("Only PREDICT modes are supported: %s" % (mode))

    tvars = tf.trainable_variables()
    scaffold_fn = None
    (assignment_map,
     initialized_variable_names) = modeling.get_assignment_map_from_checkpoint(
         tvars, init_checkpoint)
    if use_tpu:

      def tpu_scaffold():
        tf.train.init_from_checkpoint(init_checkpoint, assignment_map)
        return tf.train.Scaffold()

      scaffold_fn = tpu_scaffold
    else:
      tf.train.init_from_checkpoint(init_checkpoint, assignment_map)

    tf.logging.info("**** Trainable Variables ****")
    for var in tvars:
      init_string = ""
      if var.name in initialized_variable_names:
        init_string = ", *INIT_FROM_CKPT*"
      tf.logging.info("  name = %s, shape = %s%s", var.name, var.shape,
                      init_string)

    encoder_layer = model.all_encoder_layers[LAYERS[1]]
    
    mul_mask = lambda x, m: x * tf.expand_dims(m, axis=-1)
    masked_reduce_mean = lambda x, m: tf.reduce_sum(mul_mask(x, m), axis=1) / (
            tf.reduce_sum(m, axis=1, keepdims=True) + 1e-10)

    input_mask = tf.cast(input_mask, tf.float32)
    
    # 以下代码是句向量的生成方法，可以理解为做了一个卷积的操作，但是没有把结果相加, 卷积核是input_mask
    pooled = masked_reduce_mean(encoder_layer, input_mask)
#     pooled = tf.identity(pooled, 'final_encodes')
#     print(pooled.shape)

#     output_tensors = [pooled]

    predictions = {
        "unique_id": unique_ids,
    }

#     for (i, layer_index) in enumerate(layer_indexes):
#       predictions["layer_output_%d" % i] = all_layers[layer_index]
    predictions["encode"]= pooled
    output_spec = tf.contrib.tpu.TPUEstimatorSpec(
        mode=mode, predictions=predictions, scaffold_fn=scaffold_fn)
    return output_spec

  return model_fn

convert tokens to id

In [0]:
def convert_to_ids(tokens):
  vectors = [dictionary.get(i) if i in dictionary else dictionary.get('[UNK]') for i in tokens]
  return vectors

In [0]:
def convert_examples_to_features(examples, seq_length, tokenizer):
  """Loads a data file into a list of `InputBatch`s."""

  features = []
  for (ex_index, example) in enumerate(examples):
    tokens_a = tokenizer.tokenize(example.text_a)
    tokens_b = None

    if len(tokens_a) > seq_length - 2:
      tokens_a = tokens_a[0:(seq_length - 2)]

    # The convention in BERT is:
    # (a) For sequence pairs:
    #  tokens:   [CLS] is this jack ##son ##ville ? [SEP] no it is not . [SEP]
    #  type_ids: 0     0  0    0    0     0       0 0     1  1  1  1   1 1
    # (b) For single sequences:
    #  tokens:   [CLS] the dog is hairy . [SEP]
    #  type_ids: 0     0   0   0  0     0 0
    #
    # Where "type_ids" are used to indicate whether this is the first
    # sequence or the second sequence. The embedding vectors for `type=0` and
    # `type=1` were learned during pre-training and are added to the wordpiece
    # embedding vector (and position vector). This is not *strictly* necessary
    # since the [SEP] token unambiguously separates the sequences, but it makes
    # it easier for the model to learn the concept of sequences.
    #
    # For classification tasks, the first vector (corresponding to [CLS]) is
    # used as as the "sentence vector". Note that this only makes sense because
    # the entire model is fine-tuned.
    tokens_a.insert(0,"[CLS]")
    input_type_ids =[0]*seq_length
    tokens_a.append("[SEP]")


    input_ids = convert_to_ids(tokens_a)

    # The mask has 1 for real tokens and 0 for padding tokens. Only real
    # tokens are attended to.
    input_mask = [1] * len(input_ids)
    
    if len(input_ids) < seq_length:
      gap = seq_length -len(input_ids)
      input_ids.extend([0] * gap )
      input_mask.extend([0] * gap )
      
    features.append(
        InputFeatures(
            unique_id=example.unique_id,
            tokens=tokens_a,
            input_ids=input_ids,
            input_mask=input_mask,
            input_type_ids=input_type_ids))
  return features

In [0]:
def _truncate_seq_pair(tokens_a, tokens_b, max_length):
  """Truncates a sequence pair in place to the maximum length."""

  # This is a simple heuristic which will always truncate the longer sequence
  # one token at a time. This makes more sense than truncating an equal percent
  # of tokens from each, since if one sequence is very short then each token
  # that's truncated likely contains more information than a longer sequence.
  while True:
    total_length = len(tokens_a) + len(tokens_b)
    if total_length <= max_length:
      break
    if len(tokens_a) > len(tokens_b):
      tokens_a.pop()
    else:
      tokens_b.pop()

In [0]:
def read_sequence(input_sentences):
  examples = []
  unique_id = 0
  for sentence in input_sentences:
    line = tokenization.convert_to_unicode(sentence)
    examples.append(InputExample(unique_id=unique_id, text_a=line))
    unique_id += 1
  return examples
  


Get features from tweets

In [0]:
def get_features(input_text, fp = None, dim=1024):
#   tf.logging.set_verbosity(tf.logging.INFO)

  layer_indexes = LAYERS

  bert_config = modeling.BertConfig.from_json_file(BERT_CONFIG)

  tokenizer = tokenization.FullTokenizer(
       vocab_file=VOCAB_FILE, do_lower_case=True)

  is_per_host = tf.contrib.tpu.InputPipelineConfig.PER_HOST_V2
  tpu_cluster_resolver = tf.contrib.cluster_resolver.TPUClusterResolver(TPU_ADDRESS)
  run_config = tf.contrib.tpu.RunConfig(
      cluster=tpu_cluster_resolver,
      tpu_config=tf.contrib.tpu.TPUConfig(
          num_shards=NUM_TPU_CORES,
          per_host_input_for_training=is_per_host))

  examples = read_sequence(input_text)

  features = convert_examples_to_features(
      examples=examples, seq_length=MAX_SEQ_LENGTH, tokenizer=mytokenizer)

  unique_id_to_feature = {}
  for feature in features:
    unique_id_to_feature[feature.unique_id] = feature

  model_fn = model_fn_builder(
      bert_config=bert_config,
      init_checkpoint=INIT_CHECKPOINT,
      layer_indexes=layer_indexes,
      use_tpu=True,
      use_one_hot_embeddings=False)

  # If TPU is not available, this will fall back to normal Estimator on CPU
  # or GPU.
  estimator = tf.contrib.tpu.TPUEstimator(
      use_tpu=True,
      model_fn=model_fn,
      config=run_config,
      predict_batch_size=BATCH_SIZE,
      train_batch_size=BATCH_SIZE)

  input_fn = input_fn_builder(
      features=features, seq_length=MAX_SEQ_LENGTH)
  output = []
  # Get features
  for i_count, result in enumerate(estimator.predict(input_fn, yield_single_examples=True)):
    unique_id = int(result["unique_id"])
    feature = unique_id_to_feature[unique_id]
    if fp is None:
      output.append(result["encode"])
    else:
      #print(result["layer_output_1"])
      json.dump(result["layer_output_1"].tolist(), fp)
    if i_count % 10000 == 0:
      print(i_count)
#     output = collections.OrderedDict()
#     for (i, token) in enumerate(feature.tokens):
#       layers = []
#       for (j, layer_index) in enumerate(layer_indexes):
#         layer_output = result["layer_output_%d" % j]
#         layer_output_flat = np.array([x for x in layer_output[i:(i + 1)].flat])
#         layers.append(layer_output_flat)
        
#       output[token] = sum(layers)[:dim]
  return output

Prepare data for training and testing

In [18]:
from sklearn.model_selection import train_test_split
def get_data():
    X = []
    Y = []
    with open ('/content/gdrive/My Drive/SML_Project1/train_tweets.txt') as fp:
        for line in fp:
            data = line.split("\t")
            X.append(data[1])
            Y.append(int(data[0]))
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.1, random_state=90051)
    '''
    with open ('/content/gdrive/My Drive/SML_Project1/test_tweets_unlabeled.txt') as fp:
        for line in fp:
            X_test.append(data[1])
    '''
    return X_train, Y_train, X_test, Y_test

X_train, Y_train, X_test, Y_test = get_data()

real_to_label = {}
label_to_real = {}
Y_train_label = []
Y_test_label = []
for user in Y_train:
    if user not in real_to_label.keys():
        new_label = len(real_to_label)
        real_to_label[user] = len(real_to_label)
        label_to_real[new_label] = user
    Y_train_label.append(real_to_label[user])
for user in Y_test:
    if user not in real_to_label.keys():
        new_label = len(real_to_label)
        real_to_label[user] = len(real_to_label)
        label_to_real[new_label] = user
    Y_test_label.append(real_to_label[user])

X_final_test = []
with open ('/content/gdrive/My Drive/SML_Project1/test_tweets_unlabeled.txt') as fp:
    for line in fp:
        X_final_test.append(line)
    
print(len(X_train))
print(len(Y_train))
print(len(Y_train_label))
print(len(X_test))
print(len(Y_test))
print(len(Y_test_label))
print(len(X_final_test))
print(len(real_to_label))
print(len(label_to_real))

296038
296038
296038
32894
32894
32894
35437
9297
9297


Embedding

In [19]:
#import json
#with open("/content/gdrive/My Drive/SML_Project1/x_train_encode.json", "a") as fp:
#  X_train_encode = get_features(X_train, fp)
X_train_encode = get_features(X_train)
X_test_encode = get_features(X_test)
X_final_test_encode = get_features(X_final_test)

print(mytokenizer.get_max_seq())
print(len(X_train_encode))
print(len(X_train_encode[0]))
print(len(X_test_encode))
print(len(X_test_encode[0]))
print(len(X_final_test_encode))
print(len(X_final_test_encode[0]))

INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmp5cbqczw2', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
cluster_def {
  job {
    name: "worker"
    tasks {
      key: 0
      value: "10.43.48.66:8470"
    }
  }
}
isolate_session_state: true
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': None, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7ff3bdda6c50>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': 'grpc://10.43.48.66:8470', '_evaluation_master': 'grpc://10.43.48.66:8470', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1, '_tpu_config': TPUConfig(i

Get other fetures into vectors

In [20]:
other_features = np.array(mytokenizer.get_other_features())
additional_info = other_features[:len(X_train_encode), :]
additional_info_val = other_features[len(X_train_encode) : len(X_train_encode) + len(X_test_encode), :]
additional_info_test = other_features[-len(X_final_test_encode) : , :]
print(len(other_features))
print(len(additional_info))
print(len(additional_info[0]))
print(len(additional_info_val))
print(len(additional_info_val[0]))
print(len(additional_info_test))
print(len(additional_info_test[0]))

364369
296038
8
32894
8
35437
8


In [21]:
import keras
num_classes = len(real_to_label)
x_train = X_train_encode# 3D tensor
y_train = Y_train_label #a int list [labels]
x_val = X_test_encode
y_val = Y_test_label
y_train_labels = keras.utils.to_categorical(y_train, num_classes=num_classes)  # to one-hot metrics
y_val_labels = keras.utils.to_categorical(y_val, num_classes=num_classes)

Using TensorFlow backend.


Picture of training

In [0]:
from keras.callbacks import Callback
import matplotlib.pyplot as plt
import matplotlib
matplotlib.use('Agg')

class LossHistory(Callback):
    def on_train_begin(self, logs={}):
        self.losses = {'batch':[], 'epoch':[]}
        self.accuracy = {'batch':[], 'epoch':[]}
        self.val_loss = {'batch':[], 'epoch':[]}
        self.val_acc = {'batch':[], 'epoch':[]}

    def on_batch_end(self, batch, logs={}):
        self.losses['batch'].append(logs.get('loss'))
        self.accuracy['batch'].append(logs.get('acc'))
        self.val_loss['batch'].append(logs.get('val_loss'))
        self.val_acc['batch'].append(logs.get('val_acc'))

    def on_epoch_end(self, batch, logs={}):
        self.losses['epoch'].append(logs.get('loss'))
        self.accuracy['epoch'].append(logs.get('acc'))
        self.val_loss['epoch'].append(logs.get('val_loss'))
        self.val_acc['epoch'].append(logs.get('val_acc'))

    def loss_plot(self, loss_type, savepath):
        iters = range(len(self.losses[loss_type]))
        plt.figure()
        # acc
        plt.plot(iters, self.accuracy[loss_type], 'r', label='train acc')
        # loss
        plt.plot(iters, self.losses[loss_type], 'g', label='train loss')
        # val_acc
        plt.plot(iters, self.val_acc[loss_type], 'b', label='val acc')
        # val_loss
        plt.plot(iters, self.val_loss[loss_type], 'k', label='val loss')
        plt.grid(True)
        plt.xlabel(loss_type)
        plt.ylabel('acc-loss')
        plt.legend(loc="upper right")    
        plt.savefig(savepath)

Model

In [24]:
from keras.models import Model
from keras.layers import Input, Flatten, Dropout, concatenate, Dense, Activation, concatenate
from keras.optimizers import SGD, Adam
from keras import regularizers
from keras.callbacks import ModelCheckpoint, EarlyStopping

def MLP(auxilury_input_size = (2, ), main_input_size = (768, ), num_classes = 10001):
    
    auxilury_input = Input(shape=auxilury_input_size)
    main_input = Input(shape=main_input_size, dtype='float32')
    
    all_input = concatenate([main_input, auxilury_input], axis=-1)

    print(auxilury_input_size[0] + main_input_size[0])
#    dense1 = Dense(64, activation='relu', input_dim=auxilury_input_size[0] + main_input_size[0], kernel_regularizer=regularizers.l2(1e-5))(all_input)
#    drop1 = Dropout(0.01)(dense1)
#    dense2 = Dense(64, activation='relu', kernel_regularizer=regularizers.l2(1e-5))(dense1)
#    drop2 = Dropout(0.01)(dense2)
    main_output = Dense(num_classes, activation='softmax', kernel_regularizer=regularizers.l2(1e-7))(all_input)
#     main_output = Dense(num_classes, activation='softmax')(all_input)

    #compile the model
    model = Model(inputs=[main_input, auxilury_input], outputs=main_output)
    sgd = SGD(lr=0.01, decay=1e-7, momentum=0.9, nesterov=True)
    #sgd = SGD(lr=0.005, momentum=0.9)
    #adam = Adam(lr = 0.006)
    model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
    
    return model

model_path = '/content/gdrive/My Drive/SML_Project1/mlp_sgd.hdf5'
auxilury_input_size = (len(additional_info[0]),)
main_input_size = (len(X_train_encode[0]),)

history = LossHistory()
model = MLP(auxilury_input_size = auxilury_input_size, main_input_size = main_input_size, num_classes = num_classes)
model_checkpoint = ModelCheckpoint(model_path, monitor='loss', verbose=1, save_best_only=True)
model.fit([x_train, additional_info], y_train_labels, batch_size=32, epochs=20, validation_data=([x_val, additional_info_val], y_val_labels), 
          callbacks=[model_checkpoint, EarlyStopping(monitor="val_loss", patience=3), history])
result_image_path = '/content/gdrive/My Drive/SML_Project1/mlp_sgd.png'
history.loss_plot('epoch', result_image_path)

776
Train on 296038 samples, validate on 32894 samples
Epoch 1/20
296038/296038 [==============================] - 787s 3ms/step - loss: 8.5986 - acc: 0.0225 - val_loss: 8.2695 - val_acc: 0.0344

Epoch 00001: loss improved from inf to 8.59864, saving model to /content/gdrive/My Drive/SML_Project1/mlp_sgd.hdf5
Epoch 2/20
296038/296038 [==============================] - 777s 3ms/step - loss: 7.9048 - acc: 0.0546 - val_loss: 7.8679 - val_acc: 0.0602

Epoch 00002: loss improved from 8.59864 to 7.90485, saving model to /content/gdrive/My Drive/SML_Project1/mlp_sgd.hdf5
Epoch 3/20
296038/296038 [==============================] - 751s 3ms/step - loss: 7.4780 - acc: 0.0782 - val_loss: 7.6158 - val_acc: 0.0760

Epoch 00003: loss improved from 7.90485 to 7.47799, saving model to /content/gdrive/My Drive/SML_Project1/mlp_sgd.hdf5
Epoch 4/20
296038/296038 [==============================] - 758s 3ms/step - loss: 7.1701 - acc: 0.0972 - val_loss: 7.4446 - val_acc: 0.0887

Epoch 00004: loss improved f

Continue training if the training stopped by Google

In [0]:
from keras.models import load_model
from keras.models import Model
from keras.layers import Input, Flatten, Dropout, concatenate, Dense, Activation, concatenate
from keras.optimizers import SGD, Adam
from keras import regularizers
from keras.callbacks import ModelCheckpoint, EarlyStopping

history = LossHistory()
model_path = '/content/gdrive/My Drive/SML_Project1/mlp_sgd.hdf5'
model = load_model(model_path)
model_checkpoint = ModelCheckpoint(model_path, monitor='loss', verbose=1, save_best_only=True)
model.fit([x_train, additional_info], y_train_labels, batch_size=32, epochs=50, validation_data=([x_val, additional_info_val], y_val_labels), 
          callbacks=[model_checkpoint, EarlyStopping(monitor="val_loss", patience=3), history])
result_image_path = '/content/gdrive/My Drive/SML_Project1/mlp_sgd_40+.png'
history.loss_plot('epoch', result_image_path)

296038/296038 [==============================] - 591s 2ms/step - loss: 4.1161 - acc: 0.3532 - val_loss: 6.2210 - val_acc: 0.1782

Epoch 00001: loss improved from inf to 4.11614, saving model to /content/gdrive/My Drive/SML_Project1/mlp_sgd.hdf5

Epoch 00001: loss improved from inf to 4.11614, saving model to /content/gdrive/My Drive/SML_Project1/mlp_sgd.hdf5
Epoch 2/50
296038/296038 [==============================] - 576s 2ms/step - loss: 4.0888 - acc: 0.3562 - val_loss: 6.2180 - val_acc: 0.1796

Epoch 00002: loss improved from 4.11614 to 4.08882, saving model to /content/gdrive/My Drive/SML_Project1/mlp_sgd.hdf5

Epoch 00002: loss improved from 4.11614 to 4.08882, saving model to /content/gdrive/My Drive/SML_Project1/mlp_sgd.hdf5
Epoch 3/50
296038/296038 [==============================] - 575s 2ms/step - loss: 4.0616 - acc: 0.3595 - val_loss: 6.2151 - val_acc: 0.1803

Epoch 00003: loss improved from 4.08882 to 4.06163, saving model to /content/gdrive/My Drive/SML_Project1/mlp_sgd.hdf5

Prediction

In [0]:
result = model.predict([X_final_test_encode, additional_info_test], verbose=1)
print(len(result))
print(len(result[0]))

35437/35437 [==============================] - 13s 365us/step
35437
9297


In [0]:
test_users = []
for res in result:
    t_list = res.tolist()
    test_users.append(label_to_real[t_list.index(max(t_list))])
print(len(test_users))
print(test_users[0])

35437
7952


Store in csv

In [0]:
import pandas as pd
output_file = "/content/gdrive/My Drive/SML_Project1/pridicted.csv"
df = pd.DataFrame({'Id': np.arange(1, len(test_users) + 1),
                  'Predicted' : np.array(test_users)})
print(df)
df.to_csv(output_file,index=False)

          Id  Predicted
0          1       7952
1          2       4132
2          3       6492
3          4       5364
4          5       8297
5          6        164
6          7       9016
7          8       5573
8          9       3029
9         10       6407
10        11       8739
11        12       1624
12        13       5698
13        14       8456
14        15       5801
15        16       5140
16        17       8760
17        18       8769
18        19       9398
19        20       9022
20        21       1966
21        22       5647
22        23       7647
23        24       8704
24        25       6986
25        26       6094
26        27       7952
27        28       8739
28        29       9251
29        30       8568
...      ...        ...
35407  35408       9287
35408  35409       3234
35409  35410       5343
35410  35411       3794
35411  35412       7170
35412  35413       2911
35413  35414       5519
35414  35415       5523
35415  35416       8057
35416  35417    